In [1]:
#Importing libraries
from bs4 import BeautifulSoup
import requests
import lxml
import re
import IPython
import os
import urllib
import time
import random
import pymongo
from pymongo import MongoClient
import numpy as np
import json

####  Search on yelp.com for top-40 “Donut Shops” in the San Francisco area & Saving each search result page to disk

In [2]:
donut_yelp="https://www.yelp.com/search?find_desc=donut+shop&find_loc=San+Francisco%2C+CA+94105"
header = {"user-agent": "Mozilla/5.0 (X11; Linux x86_64; rv:12.0) Gecko/20100101 Firefox/12.0"}

In [3]:
#Getting the page into a variable and creating a new file
page_yelp = requests.get(url = donut_yelp, headers = header) 

with open("yelp_donut.htm", 'wb') as fd:
            fd.write(page_yelp.content)

In [4]:
pg_no = ["0","10","20","30"]
webpages_lg = list(range(1,5))

#Appending the page number and saving it into a new url.
for i in pg_no:
    url2 =f"{donut_yelp}&start={i}"
    donut_page2 = requests.get(url = url2, headers = header)
    #Writing each page into a html file
    with open(f"sf_donut_shop_search_page_{pg_no.index(i)}.html", 'wb') as yelp_html:
            yelp_html.write(donut_page2.content)
    #Printing how many pages are scraped
    print(f"Page {pg_no.index(i)} scraped.")
    time.sleep(10)

Page 0 scraped.
Page 1 scraped.
Page 2 scraped.
Page 3 scraped.


#### Parsing out all shop information (search rank, name, linked URL [this store’s Yelp URL], star rating, number of reviews, store tags, “$” signs, delivery / dine-in tags, and whether you can order through Yelp). Please be sure to skip all “Sponsored Results” and creating a MongoDB collection called “sf_donut_shops” that stores all the extracted shop information, one document for each shop.

In [5]:
#Creating a mongo db collection containingg all the html files and their contents
client =  MongoClient('mongodb://localhost:27017/?readPreference=primary&appname=MongoDB%20Compass&directConnection=true&ssl=false')
database = client["sf_donut_shops"]
coll = database["sf_donut_shops"]
#Creating an empty list to save all the url links 
urlsList = []
#Each item in the above list stores an output from the html file
for i in range(0,4):
    with open("sf_donut_shop_search_page_"+ str(i) + ".html", encoding="utf8") as file:
        contents = file.read()

    #Parsing the html
    donuts_parsed = BeautifulSoup(contents,'lxml')
    
    #Selecting a class in order to check for the required parameters present inside this class
    tags = donuts_parsed.select('div[class*="hoverable"]')

    #Run a loop to print all item details on the page while skipping the sponsored results (we see that in every page, the first two and the last shops are sponsored)
    for i in range(2,12):
        tag = tags[i]
        
        #Obtaining each store name and storing in a variable
        product_name=tag.find_all('h3')[0].text
        rank = tag.find('h3').find('span').get_text(separator=" ").split(".")[0]
        
        #Obtaining all categories to which the shops belong such as "Bakeries",Donuts',etc and storing it in a variable
        categories=tag.findAll('span',{'class':'css-epvm6 display--inline__09f24__c6N_k border-color--default__09f24__NPAKY'})[0].text
        categories_seperate=re.findall('[A-Z][^A-Z]*', categories)
        abc=" ".join(map(str,categories_seperate))
        
        try:
            #Obtaining all the types present such as 'Outdoor dining', 'Delivery' & 'Takeout' using regex and storing it in a variable
            types_=tag.findAll('div',{'data-testid': 'TRUSTED_PROPERTY'})[0].text
            seperate_types= re.findall('[A-Z][^A-Z]*', types_)
            
            #Creating an empty list to store the availibity of the above types 
            delTagVal=[]
            if tag.find("div", {"data-testid": "TRUSTED_PROPERTY"}):
                mydivs = tag.find("div", {"data-testid": "TRUSTED_PROPERTY"})
                valTags = mydivs.find_all("span", {"role": "img"})
                
                #Creating boolean naming for the availability of types ('Yes' for availability and 'No' for non-availbility)
                for i in range(len(valTags)):
                    if mydivs.select('path[d*="M9.06 8l3.47-3.47a.75.75 0 00-1.06-1.06L8 6.94 4.53 3.47a.75.75 0 00-1.06 1.06L6.94 8l-3.47 3.47a.75.75 0 101.06 1.06L8 9.06l3.47 3.47a.75.75 0 001.06-1.06L9.06 8z"]'):
                        delTagVal.append('No')
                    elif mydivs.select('path[d*="M6.308 11.763a.748.748 0 01-.53-.22l-2.641-2.64a.75.75 0 011.06-1.061l2.11 2.11 5.496-5.495a.75.75 0 111.06 1.06l-6.025 6.026a.748.748 0 01-.53.22z"]'):
                        delTagVal.append('Yes')
            dictionary_type=np.hstack(zip(delTagVal,seperate_types))
            abc_type=" ".join(map(str,dictionary_type))
        except:
            dictionary_type=" "
        
        #Obtaining the count of reviews given for each store
        review_count=tag.findAll('span',{'class':'reviewCount__09f24__tnBk4 css-1e4fdj9'})[0].text
        reviews="".join(map(str,review_count))
        
        #Obtaining the rating value for each store using regex
        regex2 = re.compile('.i-stars.')
        rating = tag.find("div", {"class" : regex2})['aria-label']
        
        #Checking whether there is a possibility of 'placing an order' from the website or not
        try:
            can_place_order=tag.findAll('span',{'class':'css-1enow5j'})[0].text
        except:
            can_place_order=' '
           
        #Obtaining the price tag for each store ( $,$$,$$$)
        try:
            price_range=tag.findAll('span',{'class':'priceRange__09f24__mmOuH css-18qxe2r'})[0].text
        except:
            price_range=' '
        
        #Storing each store's URL in a variable
        product_url="https://www.yelp.com"+tag.a.get('href')
        
        #Creating key-value pairs to store all the variables and their values in a dictionary format
        var=["ProductName","Categories","Type","Reviews","Ratings","can_place_order","PriceRange","Product URL","Rank"]
        value=[product_name,abc,abc_type,reviews,rating,can_place_order,price_range,product_url,int(rank)]
        
        ab=dict(zip(var,value))
        
        #Displaying all the details regarding the stores in yelp
        print("Product Name:",product_name)
        print("Categories:",abc)
        print("Type:",abc_type)
        print("Reviews:",reviews)
        print("Ratings:",rating)
        print("Can place order:",can_place_order)
        print("Price Range:",price_range)
        print("Product URL:",product_url)
        print("Rank:",rank)
        print("\n")
        
        #Adding the obtainined dictionary into the MongoDB collection
        coll.insert_one(ab)

        #Creating a list containing only the Product URLs
        array = list(coll.find({},{'_id':0,'Product URL': 1}))
        hy=ab.get('Product URL')
        
        #Appending the URLs to the empty list created
        urlsList.append(hy)

print(urlsList)

<ipython-input-5-cb600c8ac7b0>:45: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  dictionary_type=np.hstack(zip(delTagVal,seperate_types))


Product Name: 1. Donut Farm
Categories: Donuts Vegan
Type: Yes Takeout
Reviews: 319
Ratings: 4 star rating
Can place order:  
Price Range: $
Product URL: https://www.yelp.com/biz/donut-farm-san-francisco?osq=donut+shop
Rank: 1 


Product Name: 2. Happy Donuts
Categories: Donuts Vietnamese Thai
Type: Yes Takeout
Reviews: 218
Ratings: 3.5 star rating
Can place order:  
Price Range: $
Product URL: https://www.yelp.com/biz/happy-donuts-san-francisco-19?osq=donut+shop
Rank: 2 


Product Name: 3. Happy Donut
Categories: Donuts Coffee &  Tea Sandwiches
Type: Yes Takeout
Reviews: 185
Ratings: 4 star rating
Can place order:  
Price Range: $
Product URL: https://www.yelp.com/biz/happy-donut-san-francisco-7?osq=donut+shop
Rank: 3 


Product Name: 4. Happy Donut
Categories: Donuts
Type: Yes Takeout
Reviews: 42
Ratings: 4 star rating
Can place order:  
Price Range: $
Product URL: https://www.yelp.com/biz/happy-donut-san-francisco-8?osq=donut+shop
Rank: 4 


Product Name: 5. Hookt Doughnuts
Categori

#### Reading all URLs stored in “sf_donut_shops” and downloading each shop page

In [6]:
prods_index=list(range(0,40))
for i in urlsList:
    url2 =i
    yelp2 = requests.get(url = url2, headers = header)
    
    #Writing each donut shop into a seperate html file and storing it into the local
    with open(f"sf_donut_shop_{urlsList.index(i)}.html", 'wb') as yelpdonuts_html:
        yelpdonuts_html.write(yelp2.content)

    print(f"Donut_shop {urlsList.index(i)} scraped.")
    time.sleep(5)

Donut_shop 0 scraped.
Donut_shop 1 scraped.
Donut_shop 2 scraped.
Donut_shop 3 scraped.
Donut_shop 4 scraped.
Donut_shop 5 scraped.
Donut_shop 6 scraped.
Donut_shop 7 scraped.
Donut_shop 8 scraped.
Donut_shop 9 scraped.
Donut_shop 10 scraped.
Donut_shop 11 scraped.
Donut_shop 12 scraped.
Donut_shop 13 scraped.
Donut_shop 14 scraped.
Donut_shop 15 scraped.
Donut_shop 16 scraped.
Donut_shop 17 scraped.
Donut_shop 18 scraped.
Donut_shop 19 scraped.
Donut_shop 20 scraped.
Donut_shop 21 scraped.
Donut_shop 22 scraped.
Donut_shop 23 scraped.
Donut_shop 24 scraped.
Donut_shop 25 scraped.
Donut_shop 26 scraped.
Donut_shop 27 scraped.
Donut_shop 28 scraped.
Donut_shop 29 scraped.
Donut_shop 30 scraped.
Donut_shop 31 scraped.
Donut_shop 32 scraped.
Donut_shop 33 scraped.
Donut_shop 34 scraped.
Donut_shop 35 scraped.
Donut_shop 36 scraped.
Donut_shop 37 scraped.
Donut_shop 38 scraped.
Donut_shop 39 scraped.


#### Reading the 40 shop pages and parsing each shop’s address, phone number, and website. Signing up for a free account with https://positionstack.com/ and querying each shop address’ geolocation (long, lat). Then, updating each shop document on the MongoDB collection “sf_donut_shops” to contain the shop’s address, phone number, website, and geolocation.

In [10]:
#Creating an empty list to store the address, phone number and website of each shop
shop_details={"address":[],"phone_number":[],"website":[]}

for i in urlsList:
    
    #Reading each html file created above for each shop
    with open(f"sf_donut_shop_{urlsList.index(i)}.html", encoding="utf8") as file:
        contents = file.read()
        
    #Parsing the html files
    soup = BeautifulSoup(contents,'lxml')
    rank = urlsList.index(i) + 1
    
    #Selecting a class in order to check for the required parameters present inside this class
    xz = soup.findAll("div",{"class":"arrange__09f24__LDfbs gutter-2__09f24__CCmUo vertical-align-middle__09f24__zU9sE border-color--default__09f24__NPAKY"})
    
    #Creating a cariable to identify only the text part of the source code
    info_str=[i.text for i in xz]
    
    #Obtaining and saving each website info
    try:
        #Since every website url has a 'Business website' in the <p> section, identifying that first so that it would be easier to display and store the url
        website_id=[('Business website' in i) for i in info_str].index(True)
        
        #Once the 'Business website' is identified, replacing it with blank so that the first text would be the actual phone number
        website=info_str[website_id].replace("Business website","")
        
    except:
        website=''
    
    #Updating the website url in the mongoDB collection
    coll.update_one({"Rank":rank}, {"$set":{"Website":website}})
    
    #Obtaining and saving each shop's phone number
    try:
        #Since every website url has a 'Phone number' in the <p> section, identifying that first so that it would be easier to display and store the phone number
        phone_number_id=[('Phone number' in i) for i in info_str].index(True)
        
        #Once the 'Phone number' is identified, replacing it with blank so that the first text would be the actual phone number
        phone_number=info_str[phone_number_id].replace("Phone number","")
    except:
        phone_number=''
    
    #Updating the phone number in the mongoDB collection
    coll.update_one({"Rank":rank}, {"$set":{"Phone number":phone_number}})
    
    #Obtaining and saving each shop's address
    try:
        #Since every website url has a 'Get Directions' in the <p> section, identifying that first so that it would be easier to display and store the address
        address_id = [('Get Directions' in i) for i in info_str].index(True)
        
        #Once the 'Get Directions' is identified, replacing it with blank so that the first text would be the actual address
        address = info_str[address_id].replace("Get Directions","")
    except:
        address =''
    
    #Updating the address in the mongoDB collection
    coll.update_one({"Rank":rank}, {"$set":{"Address":address}})
    
    try:
        #Obtaining the get requests for each of the address
        apiurl = "http://api.positionstack.com/v1/forward?access_key=2f1d2c59ea97e993ce1b6d72a0e14e41&query="+ address
        rec = requests.get(apiurl)
        location = rec.json()

    
    #Selectiong only the first part of the file since that contains the latitude nad longitude of each address
        latitude = location['data'][0]['latitude']
        longitude=location['data'][0]['longitude']
    
    #Adding each of the latitude and longitude value for each address in the list created above
        geo_loc = [latitude,longitude]
        coll.update_one({"Rank":rank}, {"$set":{"Geo Location":geo_loc}})
        print(geo_loc)
    except:
        pass


[37.796258, -122.393871]
[37.791569, -122.399964]
[37.796872, -122.405494]
[37.784894, -122.410896]
[37.778008, -122.431272]
[37.778008, -122.431272]
[37.795631, -122.393177]
[37.786971, -122.399577]
[37.775238, -122.417523]
[37.786208, -122.40734]
[37.796258, -122.393871]
[37.785751, -122.412393]
[37.776285, -122.415086]
[37.790392, -122.417046]
[37.7898, -122.405018]
[37.796333, -122.407032]
[37.778008, -122.431272]
[37.781768, -122.416851]
[37.775217, -122.419519]
[37.778008, -122.431272]
[37.777725, -122.395975]
[37.807284, -122.416448]
[37.781928, -122.415341]
[37.78247, -122.407756]
[37.783941, -122.407479]
[37.798877, -122.408556]
[37.799183, -122.4083]
[37.794238, -122.404549]
[37.798081, -122.407304]
[37.778008, -122.431272]
[37.790003, -122.400907]
[37.798112, -122.408494]
[37.771306, -122.408759]
[37.795533, -122.40848]
[37.786664, -122.415095]
[37.778008, -122.431272]
[37.796099, -122.405482]
[37.778008, -122.431272]


In [11]:
#Creating an index for each shop
coll.create_index('Rank')
print(coll.index_information())

{'_id_': {'v': 2, 'key': [('_id', 1)]}, 'Rank_1': {'v': 2, 'key': [('Rank', 1)]}}


In [12]:
#Final collection
print(list(coll.find({})))

[{'_id': ObjectId('6221d88389b7055f608f4122'), 'ProductName': '1.\xa0Donut Farm', 'Categories': 'Donuts Vegan', 'Type': 'Yes Takeout', 'Reviews': '319', 'Ratings': '4 star rating', 'can_place_order': ' ', 'PriceRange': '$', 'Product URL': 'https://www.yelp.com/biz/donut-farm-san-francisco?osq=donut+shop', 'Rank': 1, 'Website': 'https://www.vegandonut.farm', 'Phone number': '', 'Address': '1 Ferry Bldg Ste 14 San Francisco, CA 94111', 'Geo Location': [37.796258, -122.393871]}, {'_id': ObjectId('6221d88389b7055f608f4123'), 'ProductName': '2.\xa0Happy Donuts', 'Categories': 'Donuts Vietnamese Thai', 'Type': 'Yes Takeout', 'Reviews': '218', 'Ratings': '3.5 star rating', 'can_place_order': ' ', 'PriceRange': '$', 'Product URL': 'https://www.yelp.com/biz/happy-donuts-san-francisco-19?osq=donut+shop', 'Rank': 2, 'Website': '', 'Phone number': '(415) 398-6769', 'Address': '100 Bush St San Francisco, CA 94111', 'Geo Location': [37.791569, -122.399964]}, {'_id': ObjectId('6221d88389b7055f608f412